In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ParameterGrid

In [2]:
# Set parameters for linear regression

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 500, num = 10)]
# Number of features to consider at every split
max_features = [1.0, 'sqrt', 'log2']
# Maximum number of levels in tree
#max_depth = [int(x) for x in np.linspace(10, 110, num = 5)]
#max_depth.append(None)
# Minimum number of samples required to split a node
#min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

warm_start = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               #'max_depth': max_depth,
               #'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'warm_start': warm_start}

grid = ParameterGrid(random_grid)
cnt = 0
for p in grid:
    cnt = cnt+1

print('Total Possible Models',cnt)


Total Possible Models 360


In [3]:
#Define MASE Metric
def mean_absolute_scaled_error(y_true, y_pred):
    mase=0
    # Define numerator as the forecast error
    numerator = (np.abs(y_true - y_pred))

    # Define denominator as the mean absolute error of the in-sample one-step naive forecast
    y_true_ohne_1 = y_true[1:].reset_index(drop=True)
    y_true_ohne_ende = y_true[:-1].reset_index(drop=True)
    denominator = np.mean(np.abs(y_true_ohne_1 - y_true_ohne_ende))

    mase = np.mean(np.abs(numerator / denominator))

    return mase

scorer_mase= make_scorer(mean_absolute_scaled_error, greater_is_better=False)


In [4]:
# Load the data
data = pd.read_parquet('/Users/paddy/Documents/GitHub/Masterthesis_ML/data/03_032022_032023_taxi_rides.parquet')

# Convert the date column to datetime
data['date'] = pd.to_datetime(data['date']) #,format='%d/%m/%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S') 


In [5]:
#Feature engineering
# Create a new column for the time
data['time'] = [x for x in range(0, len(data))]

# Drop nan values
data = data.dropna()

# Rename column count to y
data = data.rename(columns={'count': 'y'})

In [6]:
# Set y to the last column
cols = list(data.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('y')) #Remove y from list
data = data[cols+['y']] #Create new dataframe with columns in the order you want

# Define the feature columns and the target column
feature_cols = ['time']
target_col = 'y'

# drop the date column
train_data = np.delete(data, 0, 1) 

# Split the data into X and y
X_train, y_train = train_data[:, :-1], train_data[:, -1]


In [7]:
#vgl: https://lightrun.com/answers/scikit-learn-scikit-learn-grid_search-feeding-parameters-to-scorer-functions

# X and y to pandas dataframe
X_train = pd.DataFrame(X_train)
y_train = pd.Series(y_train)


In [8]:
# Set Cross Validation to 5 iterations
tscv = TimeSeriesSplit(n_splits=5)

In [9]:
# Instantiate model
model = RandomForestRegressor()

search = RandomizedSearchCV(estimator = model, 
                            param_distributions = random_grid, 
                            n_iter = 25, 
                            cv = tscv,
                            refit=True, 
                            verbose=3, 
                            random_state=42, 
                            n_jobs = -1, 
                            scoring=scorer_mase, #make_scorer(scorer_mase, greater_is_better=True), #'neg_root_mean_squared_error', #
                            error_score=np.nan)


In [10]:
# Fit the random search model
search.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END bootstrap=True, max_features=1.0, min_samples_leaf=4, n_estimators=64, warm_start=True;, score=-1.657 total time=   0.1s
[CV 2/5] END bootstrap=True, max_features=1.0, min_samples_leaf=4, n_estimators=64, warm_start=True;, score=-1.784 total time=   0.3s
[CV 1/5] END bootstrap=False, max_features=1.0, min_samples_leaf=4, n_estimators=118, warm_start=True;, score=-1.528 total time=   0.5s
[CV 3/5] END bootstrap=True, max_features=1.0, min_samples_leaf=4, n_estimators=64, warm_start=True;, score=-1.350 total time=   0.5s
[CV 2/5] END bootstrap=False, max_features=1.0, min_samples_leaf=4, n_estimators=118, warm_start=True;, score=-1.840 total time=   0.9s
[CV 4/5] END bootstrap=True, max_features=1.0, min_samples_leaf=4, n_estimators=64, warm_start=True;, score=-1.556 total time=   0.7s
[CV 1/5] END bootstrap=False, max_features=sqrt, min_samples_leaf=4, n_estimators=118, warm_start=False;, score=-1.528 total time=

RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                   estimator=RandomForestRegressor(), n_iter=25, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_features': [1.0, 'sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'n_estimators': [10, 64, 118, 173, 227,
                                                         282, 336, 391, 445,
                                                         500],
                                        'warm_start': [True, False]},
                   random_state=42,
                   scoring=make_scorer(mean_absolute_scaled_error, greater_is_better=False),
                   verbose=3)

In [11]:
print('Best Score: %s' % search.best_score_)
print('Best Hyperparameters: %s' % search.best_params_)
print('Best Model: %s' % search.best_estimator_)
print('Best Index: %s' % search.best_index_)
print('CV Results: %s' % search.cv_results_)
print('Refit Time: %s' % search.refit_time_)
print('Scorer: %s' % search.scorer_)

Best Score: -1.5204369670875604
Best Hyperparameters: {'warm_start': False, 'n_estimators': 500, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'bootstrap': True}
Best Model: RandomForestRegressor(max_features='sqrt', min_samples_leaf=4, n_estimators=500)
Best Index: 14
CV Results: {'mean_fit_time': array([2.02779546, 0.486339  , 2.08037262, 1.32738047, 3.42409482,
       5.74938121, 3.37990389, 3.26543298, 3.92532377, 1.0365417 ,
       0.53413258, 6.06410656, 4.63643656, 8.22292457, 3.58310752,
       1.37947044, 1.07814522, 0.17424164, 4.12291565, 7.24383535,
       0.06709609, 1.00940647, 2.013027  , 2.64092517, 5.45982304]), 'std_fit_time': array([1.11542172, 0.25525544, 1.15132258, 0.72756464, 1.73454914,
       2.97047304, 1.66575378, 1.72641801, 2.00144518, 0.54224748,
       0.26493313, 3.28023937, 2.50767301, 4.45639905, 1.7774692 ,
       0.77244399, 0.57741228, 0.09297086, 1.95864292, 3.99823746,
       0.03123614, 0.56425439, 1.10850167, 1.27064299, 2.54734574]), 'mean_sco